In [1]:
import os

In [2]:
%pwd

'c:\\Users\\fkokr\\Desktop\\MLProject\\Text_Summarization_Project\\research'

In [3]:
# Change root directory
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\fkokr\\Desktop\\MLProject\\Text_Summarization_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarization.constants import *
from textSummarization.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config['artifacts_root']])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']
        
        create_directories([config['root_dir']])
        
        return DataIngestionConfig(
            root_dir = config['root_dir'],
            source_URL = config['source_URL'],
            local_data_file = config['local_data_file'],
            unzip_dir = config['unzip_dir']        
        )

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarization.logging import logger
from textSummarization.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file)
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_files(self):
        """Extract zip files
        args: 
            zip_file_path[str]
        returns:
            None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_files()  
except Exception as e:
    raise e

[2024-04-23 11:02:39,956:INFO:common:YAML file: config\config.yaml loaded successfully.]
[2024-04-23 11:02:39,959:INFO:common:YAML file: params.yaml loaded successfully.]
[2024-04-23 11:02:39,961:INFO:common:Directory artifacts created successfully.]
[2024-04-23 11:02:39,962:INFO:common:Directory artifacts/data_ingestion created successfully.]
[2024-04-23 11:02:40,718:INFO:80637392:artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 8398607
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "68eed66d4647ad51826926674bd2b0fdb8afde6a0af32e0693e643107294e595"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: CF1E:36135:5B56A:63ECB:6627CD39
Accept-Ranges: bytes
Date: Tue, 23 Apr 2024 15:02:40 GMT
Via: 1.1 varnish
X-Served-By: cache-ewr18145-EWR
X-Cache